Are you looking for the second most recent activity for each user in a dataset? This is a common challenge when managing time-based records—especially if some users may have multiple entries while others have only one. In this post, we’ll explore an example problem where each user can perform activities at different times, and we need to extract their second most recent one. If a user has only one entry, we’ll simply return that. By walking through this scenario, you’ll learn practical techniques you can apply to your own data analysis tasks.

We have a table **UserActivity** with the following schema:

| Column Name | Type    |
|-------------|---------|
| username    | varchar |
| activity    | varchar |
| startDate   | Date    |
| endDate     | Date    |

- **Purpose**: We want to retrieve each user's **second most recent** activity based on their `startDate` and `endDate`.
- **Condition**: If a user only has **one** activity, we return that **single** activity.
- **Assumption**: A user **cannot** perform more than one activity at the same time (no overlapping intervals).

**Input** (UserActivity table):

| username | activity | startDate  | endDate    |
|----------|----------|------------|------------|
| Alice    | Travel   | 2020-02-12 | 2020-02-20 |
| Alice    | Dancing  | 2020-02-21 | 2020-02-23 |
| Alice    | Travel   | 2020-02-24 | 2020-02-28 |
| Bob      | Travel   | 2020-02-11 | 2020-02-18 |

**Output**:

| username | activity | startDate  | endDate    |
|----------|----------|------------|------------|
| Alice    | Dancing  | 2020-02-21 | 2020-02-23 |
| Bob      | Travel   | 2020-02-11 | 2020-02-18 |

**Explanation**:
- Alice’s most recent activity is **Travel** from `2020-02-24` to `2020-02-28`.
- Her second most recent activity (the one right before that) is **Dancing** from `2020-02-21` to `2020-02-23`.
- Bob only has one record, so we simply take his single entry.


In [1]:
import pandas as pd

data = [['Alice', 'Travel', '2020-02-12', '2020-02-20'], 
        ['Alice', 'Dancing', '2020-02-21', '2020-02-23'], 
        ['Alice', 'Travel', '2020-02-24', '2020-02-28'], 
        ['Bob', 'Travel', '2020-02-11', '2020-02-18']]
user_activity = pd.DataFrame(data, 
                             columns=['username', 
                                      'activity', 
                                      'startDate', 
                                      'endDate']).astype({'username':'object', 
                                                          'activity':'object', 
                                                          'startDate':'datetime64[ns]', 
                                                          'endDate':'datetime64[ns]'})

display(user_activity)

,username,activity,startDate,endDate
0,Alice,Travel,2020-02-12,2020-02-20
1,Alice,Dancing,2020-02-21,2020-02-23
2,Alice,Travel,2020-02-24,2020-02-28
3,Bob,Travel,2020-02-11,2020-02-18


**Step 1: Count the number of records per username**

- groupby("username") splits the data by username.
- ["startDate"].transform("count") calculates how many rows each username group has by counting the startDate column.
- The result of .transform("count") is assigned to user_activity["username_count"], giving each row the total number of rows associated with its username.

**Step 2: Rank the records for each username**
    
- Again, groupby("username") groups rows by each username.
- rank(method="dense", ascending=False) computes a rank for each row within its username group, based on the values in startDate.
- ascending=False means the row with the largest (or most recent) startDate gets rank 1, the next largest gets rank 2, and so on.
- Dense ranking means that if there is a tie, all tied rows share the same rank, and the next distinct value immediately gets the next rank (no gaps).

In [2]:
user_activity["username_count"] = user_activity.groupby(["username"])[
                                                         "startDate"].transform("count")

user_activity["rank"] = user_activity.groupby(["username"])[
                                               "startDate"].rank(method="dense",
                                                                 ascending=False)

display(user_activity)

,username,activity,startDate,endDate,username_count,rank
0,Alice,Travel,2020-02-12,2020-02-20,3,3.0
1,Alice,Dancing,2020-02-21,2020-02-23,3,2.0
2,Alice,Travel,2020-02-24,2020-02-28,3,1.0
3,Bob,Travel,2020-02-11,2020-02-18,1,1.0


**Step 3: Define conditions and filter the DataFrame**

- condition_1 checks if a row belongs to a username that appears exactly once in the DataFrame.
- condition_2 checks if a row belongs to a username that appears more than once and the row is the second-ranked (i.e., rank == 2).
- condition_1 | condition_2 (logical OR) retains rows where either the username is unique or the row is the second occurrence for usernames with multiple rows.
- Assigning this filter back to user_activity drops all other rows.



In [3]:
condition_1 = (user_activity["username_count"]==1)
condition_2 = ((user_activity["username_count"]>1) & (user_activity["rank"]==2))
user_activity = user_activity[condition_1 | condition_2]
display(user_activity)

,username,activity,startDate,endDate,username_count,rank
1,Alice,Dancing,2020-02-21,2020-02-23,3,2.0
3,Bob,Travel,2020-02-11,2020-02-18,1,1.0


**Step 4: Drop the helper columns**

- After using username_count and rank to identify and filter the desired rows, these columns are no longer needed.
- drop(columns=[...]) removes them from the DataFrame.

In [4]:
user_activity = user_activity.drop(columns=['username_count', 'rank'])
display(user_activity)

,username,activity,startDate,endDate
1,Alice,Dancing,2020-02-21,2020-02-23
3,Bob,Travel,2020-02-11,2020-02-18


References: [1] https://leetcode.com/problems/get-the-second-most-recent-activity/description/?lang=pythondata